In [1]:
import warnings
import numpy as np
import pandas as pd
import scipy.stats as stats

In [2]:
warnings.filterwarnings('ignore')

In [3]:
cancers = ['OV', 'CRC', 'BRCA']
mutations = ['missense', 'truncating', 'synonymous']

## Concat OV files from different centers

In [4]:
mut = 'missense'

In [5]:
for mut in mutations:   
    jhu = pd.read_excel('../data/Pro_regression_retro_cptac/Table.DNA.PRO.JHU.regression.linearLIMMA.ProVsMut.OV.' +
                      mut + '.xlsx')
    pnnl = pd.read_excel('../data/Pro_regression_retro_cptac/Table.DNA.PRO.PNNL.regression.linearLIMMA.ProVsMut.OV.' +
                      mut + '.xlsx')
    df = pd.concat([jhu, pnnl], axis=0)
    df.to_excel('../data/Pro_regression_retro_cptac/Table.DNA.PRO.regression.linearLIMMA.ProVsMut.OV.' + mut + '.xlsx')

## Overlap prosective and retrosepctive data

In [9]:
dat = pd.DataFrame()

for cancer in cancers:
    for mut in mutations:
#         pros = pd.read_excel('../data/DNA_RNA_regression/Table.DNA.RNA.regression.linearLIMMA.RNAVsMut.' + 
#                              cancer + '.' + mut + '.xlsx')
#         retro = pd.read_excel('../data/RNA_regression_retro_cptac/Table.DNA.RNA.regression.linearLIMMA.RNAVsMut.' + 
#                               cancer + '.' + mut + '.xlsx')
        pros = pd.read_excel('../data/DNA_PRO_regression/Table.DNA.PRO.regression.linearLIMMA.ProVsMut.' + 
                             cancer + '.' + mut + '.xlsx')
        retro = pd.read_excel('../data/Pro_regression_retro_cptac/Table.DNA.PRO.regression.linearLIMMA.ProVsMut.' + 
                              cancer + '.' + mut + '.xlsx')
        sig_pros = pros[pros['FDR'] < 0.05]
        sig_pros = sig_pros[['Gene', 'logFC', 'P.value', 'FDR']]
        sig_retro = retro[retro['FDR'] < 0.05]
        sig_retro = sig_retro[['Gene', 'logFC', 'P.value', 'FDR']]
        df = sig_pros.merge(sig_retro, on='Gene')
        df.columns = ['Gene', 'logFC_pros', 'P.value_pros', 'FDR_pros', 'logFC_retro', 'P.value_retro', 'FDR_retro']
        df['cancer'] = cancer
        df['mutation'] = mut
#         df.to_excel('../data/overlap_with_retro/eQTLs/' + cancer + mut + 'p_value.05_overlap_eQTLs.xlsx')
        df.to_excel('../data/overlap_with_retro/pQTLs/' + cancer + mut + 'p_value.05_overlap_pQTLs.xlsx')
        dat = pd.concat([dat, df], axis=0)

# dat.to_excel('../data/overlap_with_retro/eQTLs/overlap_eQTLs.xlsx')
dat.to_excel('../data/overlap_with_retro/pQTLs/overlap_pQTLs.xlsx')